In [1]:
from aesahaettr import calamity
from aesahaettr import covariances
from aesahaettr import visibilities
from aesahaettr import visualization
import copy
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/home/aaronew/science/miniconda3/envs/hera/lib/python3.8/site-packages/hera_sim/visibilities/__init__.py:22: UserWarning: HealVis failed to import.
  warnings.warn("HealVis failed to import.")
/home/aaronew/science/miniconda3/envs/hera/lib/python3.8/site-packages/hera_sim/visibilities/__init__.py:27: UserWarning: PRISim failed to import.
  warnings.warn("PRISim failed to import.")
/home/aaronew/science/miniconda3/envs/hera/lib/python3.8/site-packages/hera_sim/visibilities/__init__.py:33: UserWarning: VisGPU failed to import.
  warnings.warn("VisGPU failed to import.")
/home/aaronew/science/miniconda3/envs/hera/lib/python3.8/site-packages/hera_sim/__init__.py:23: FutureWarning: 
In the next major release, all HERA-specific variables will be removed from the codebase. The following variables will need to be accessed through new class-like structures to be introduced in the next major release: 

noise.HERA_Tsky_mdl
noise.HERA_BEAM_POLY
sigchain.HERA_NRAO_BANDPASS
rfi.HERA_RFI_STATIONS

Ad

## Free Parameters versus baseline count.

In [ ]:
nparams = []
ngains = []
ndata = []
nparams_zd = []
for nant in range(3, 12):
    evals, _ = covariances.cov_mat_simple_evecs(antenna_diameter=2.,
                                                    eigenval_cutoff=1e-10, fractional_spacing=5.,
                                                    nf=200, df=200e3, f0=150e6, antenna_count=nant,
                                                    antenna_chromaticity=2. / 3e8)
    evals_zd, _ = covariances.cov_mat_simple_evecs(antenna_diameter=2., intra_baseline_only=True,
                                                eigenval_cutoff=1e-10, fractional_spacing=5.,
                                                nf=200, df=200e3, f0=150e6, antenna_count=nant,
                                                antenna_chromaticity=2. / 3e8, )
    ngains.append(nant * 200)
    ndata.append((nant * (nant+1) / 2 - (nant-1)) * 200)
    nparams.append(len(evals))
    nparams_zd.append(len(evals_zd))

In [ ]:
plt.plot(range(3, 12), ndata, label='Ndata')
plt.plot(range(3, 12), ngains, label='Ngains')
plt.plot(range(3, 12), np.asarray(nparams) + np.asarray(ngains), label='Ngain + Nfg')
plt.plot(range(3, 12), np.asarray(nparams_zd) + np.asarray(ngains), label='Ngain + Nfg with diagonal fg modeling')

plt.legend()
plt.grid()

In [ ]:
# test fitting foregrounds
uvd_gsm, uvd_eor = visibilities.compute_visibilities(output_dir='./', nside_sky=256, antenna_count=11,
                                                       nf=200, df=200e3, f0=150e6, antenna_diameter=2.,
                                                       fractional_spacing=5, eor_fg_ratio=10**(-3.5))
uvd_total = copy.deepcopy(uvd_gsm)
uvd_total.data_array += uvd_eor.data_array

evals, evecs = covariances.cov_mat_simple_evecs(uvdata=uvd_total, antenna_diameter=2., eigenval_cutoff=1e-10, antenna_chromaticity=2. / 3e8)
# initialize with true foreground coefficients
# plus a perturbation.
data = copy.deepcopy(uvd_total.data_array.squeeze()).reshape(uvd_total.Nbls * uvd_total.Nfreqs) 
mdata = np.mean(data) * 0
data -= mdata
rmsdata = np.sqrt(np.mean(np.abs(data) ** 2.))
data /= rmsdata
fg_coeffs_true = evecs.T @ data
fg_coeffs = fg_coeffs_true + .1 * fg_coeffs_true * (np.random.randn(len(fg_coeffs_true)))

In [ ]:
weights = tf.convert_to_tensor(np.ones(data.shape, dtype=np.float64))

foreground_coefficients_real = tf.Variable(tf.convert_to_tensor(fg_coeffs.real))
foreground_coefficients_imag = tf.Variable(tf.convert_to_tensor(fg_coeffs.imag))

foreground_basis_vectors = tf.convert_to_tensor(evecs)

data_real = tf.convert_to_tensor(data.real)
data_imag = tf.convert_to_tensor(data.imag)

loss = lambda: .5 * tf.reduce_sum(tf.math.square(data_real - tf.math.reduce_sum(foreground_basis_vectors * foreground_coefficients_real, axis=1) * weights)) \
               + .5 * tf.reduce_sum(tf.math.square(data_imag - tf.math.reduce_sum(foreground_basis_vectors * foreground_coefficients_imag, axis=1) * weights))

opt = tf.optimizers.Adamax(lr=1e-2)

foreground_coefficients_real_last = foreground_coefficients_real.numpy()
foreground_coefficients_imag_last = foreground_coefficients_imag.numpy()
opt.minimize(loss, [foreground_coefficients_real, foreground_coefficients_imag])

delta_real = np.abs(foreground_coefficients_real.numpy() - foreground_coefficients_real_last).max()
delta_imag = np.abs(foreground_coefficients_imag.numpy() - foreground_coefficients_imag_last).max()




In [ ]:
var_history_real = []
var_history_imag = []
tol = 1e-5
step = 0
maxsteps = 3500
while (delta_real >= tol or delta_imag >= tol) and step < maxsteps:
    opt.minimize(loss, [foreground_coefficients_real, foreground_coefficients_imag])
    delta_real = np.abs(foreground_coefficients_real.numpy() - foreground_coefficients_real_last).max()
    delta_imag = np.abs(foreground_coefficients_imag.numpy() - foreground_coefficients_imag_last).max()
    foreground_coefficients_real_last = foreground_coefficients_real.numpy()
    var_history_real.append(foreground_coefficients_real_last)
    foreground_coefficients_imag_last = foreground_coefficients_imag.numpy()
    var_history_imag.append(foreground_coefficients_imag_last)
    step += 1

In [ ]:
step

In [ ]:
delta_imag

In [ ]:
foreground_coefficients = foreground_coefficients_real.numpy() + 1j * foreground_coefficients_imag.numpy()
foreground_model = (foreground_basis_vectors.numpy() @ foreground_coefficients).reshape(uvd_eor.Nbls, uvd_eor.Nfreqs)
foreground_model = foreground_model * rmsdata + mdata

In [ ]:
len(fg_coeffs_true)

In [ ]:
foreground_model_true = (foreground_basis_vectors.numpy() @ fg_coeffs_true).reshape(uvd_eor.Nbls, uvd_eor.Nfreqs)
foreground_model_true = foreground_model_true * rmsdata + mdata

In [ ]:
len(fg_coeffs_true) + uvd_eor.Nants_data * uvd_eor.Nfreqs

In [ ]:
len(data)

In [ ]:
delta_imag

In [ ]:
fig, axarr = plt.subplots(len(fg_coeffs[:200])//8, 8)
fig.set_size_inches(24, 68)
for i in range(len(var_history_real[0])):
    l = i % 8
    m = i // 8
    plt.sca(axarr[m][l])
    plt.plot(np.abs(np.diff(np.asarray(var_history_real).T[i])))
    plt.yscale('log')
plt.show()


In [ ]:
delta_imag

In [ ]:
fig, axarr = plt.subplots(len(fg_coeffs[:200])//8 + 1, 8)
fig.set_size_inches(48, 96)
for i in range(200):
    l = i % 8
    m = i // 8
    plt.sca(axarr[m][l])
    plt.plot(np.asarray(var_history_real).T[i])
    plt.axhline(fg_coeffs_true[i].real, ls='--', color='r')
plt.show()


In [ ]:
plt.hist(fg_coeffs_true.real - foreground_coefficients_imag_last)

In [ ]:
uvd_model = copy.deepcopy(uvd_total)
uvd_model.data_array = foreground_model.reshape(uvd_model.Nbls, 1, uvd_model.Nfreqs, 1)
uvd_resid = copy.deepcopy(uvd_total)
uvd_resid.data_array = uvd_total.data_array - uvd_model.data_array

In [ ]:
uvd_resid_true = copy.deepcopy(uvd_total)
uvd_resid_true.data_array = uvd_total.data_array -  foreground_model_true.reshape(uvd_model.Nbls, 1, uvd_model.Nfreqs, 1)

In [ ]:
uvd_resid_icov = copy.deepcopy(utd_total)
uvd_resid_icov = 

In [ ]:
mdata

In [ ]:
uvd_noise = copy.deepcopy(uvd_total)
uvd_noise.data_array = uvd_total.data_array - uvd_gsm.data_array

In [ ]:
np.median(np.abs(uvd_total.data_array) ** 2.) ** .5 / np.median(np.abs(uvd_eor.data_array) ** 2.) ** .5

In [ ]:
fig, axarr = plt.subplots(1, 4)
for i, uvt in enumerate([uvd_gsm, uvd_resid, uvd_resid_true, uvd_noise]):
    plt.sca(axarr[i])
    x, y, ctot = visualization.delay_transform_sort_by_baseline_length(uvt, window='cs11')
    xg, yg = np.meshgrid(x, y)
    plt.pcolor(xg, yg, np.log10(np.abs(ctot.T)), #extent = [x.min(), x.max(), y.max(), y.min()],
               vmin=-2, vmax=3, cmap='inferno')#, interpolation='nearest', aspect='auto', cmap='inferno')
    plt.plot(x, x / .3, ls='--', color='w')
    plt.plot(x, -x / .3, ls='--', color='w')

    plt.xlabel('b (m)')
    plt.ylabel('$\\tau$ (ns)')
    plt.colorbar()
    plt.xlim(0, 500)
    plt.ylim(-2000, 2000)
    plt.xlim()
fig.set_size_inches(18, 8)